In [397]:
import pandas as pd
import numpy as np

### Cleaning reviews.csv

#### Read File
Replace missing values with 'nan'

In [518]:
missing_values = ["n/a", "na", "--", "Nan", "NAN", "NA", "na", "X", "", "nan", "N/A"]

In [519]:
recipes = pd.read_csv('recipes.csv', encoding='latin-1', sep=';', quotechar='\'' , na_values = missing_values)

#### Check size and datatypes of the columns in recipes

In [520]:
recipes.size

184540

In [506]:
recipes.dtypes

Recipe Name     object
RecipeID        object
Review Count    object
Recipe Photo    object
Author          object
Prepare Time    object
Cook Time       object
Total Time      object
Ingredients     object
Directions      object
dtype: object

#### rename column names

In [521]:
recipes = recipes.rename(columns={"Recipe Name": "recipeName", "RecipeID": "recipeID", "Review Count": "reviewCount", "Recipe Photo": "recipePhoto", "Author": "author", "Prepare Time": "prepareTime_min", "Cook Time": "cookTime_min", "Total Time": "totalTime_min", "Ingredients" :"ingredients", "Directions" : "directions"})

In [ ]:
recipes.head(50)

#### Check if all the values in a column are of the same type

In [ ]:
for name in recipes.columns:
  count = recipes[recipes[name].map(type) != str].count()
  print(name, "-------------------------")
  print(count)


#### Convert prepareTime, cookTime and totalTime  from 'str'  to 'int'


In [510]:
#recipes.fillna("missing", inplace=True)

In [524]:
from collections import defaultdict
import re

def isNaN(string):
    return string != string

def humantime2minutes(s):
    if (isNaN(s)):
      return np.NaN
    d = {
      'w':      7*24*60,
      'week':   7*24*60,
      'weeks':  7*24*60,
      'd':      24*60,
      'day':    24*60,
      'days':   24*60,
      'h':      60,
      'hr':     60,
      'hour':   60,
      'hours':  60,
      'm': 1,
      'min': 1, 
    }
    mult_items = defaultdict(lambda: 1).copy()
    mult_items.update(d)

    parts = re.search(r'^(\d+)([^\d]*)', s.lower().replace(' ', ''))
    if parts:
        return int(parts.group(1)) * mult_items[parts.group(2)] + humantime2minutes(re.sub(r'^(\d+)([^\d]*)', '', s.lower()))
    else:
        return 0

In [525]:
recipes['prepareTime_min'] = recipes['prepareTime_min'].apply(humantime2minutes)

In [526]:
recipes['cookTime_min'] = recipes['cookTime_min'].apply(humantime2minutes)

In [527]:
recipes['totalTime_min'] = recipes['totalTime_min'].apply(humantime2minutes)

In [516]:
#recipes = recipes.replace('missing', np.NaN)

In [ ]:
recipes.head(50)

#### Numeric Data Types

###### Check if RecipeID and Review Count have only numeric values and then convert to int.


In [529]:
recipes['recipeID_is_digit'] = list(map(lambda x: not x.isdigit(), recipes['recipeID']))

In [530]:
recipes[recipes['recipeID_is_digit']]

,recipeName,recipeID,reviewCount,recipePhoto,author,prepareTime_min,cookTime_min,totalTime_min,ingredients,directions,recipeID_is_digit
245,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,0.0,0.0,0.0,Ingredients,Directions,True


In [531]:
del recipes["recipeID_is_digit"]

* ###### Record 245 is not numeric so drop it

In [532]:
recipes = recipes.drop(245)

#### Change reviewCount from str -> int
First change k to 1000 in reviewCount



In [533]:
df1 = recipes.reviewCount.replace(r'[kM]+$', '', regex=True).astype(str)

In [534]:
df2 = recipes.reviewCount.str.extract(r'[\d\.]+([km]+)', expand=False).replace('k','1000')

In [535]:
df2 = df2.fillna('')

In [536]:
recipes['reviewCount'] = df1 + df2

In [537]:
recipes['review_is_digit'] = list(map(lambda x: not x.isdigit(), recipes['reviewCount']))

In [538]:
recipes[recipes['review_is_digit']]

,recipeName,recipeID,reviewCount,recipePhoto,author,prepareTime_min,cookTime_min,totalTime_min,ingredients,directions,review_is_digit


In [539]:
del recipes["review_is_digit"]

##### Now reviewCount and recipeID have only numeric values so, convert them to int. And conver the remaining to str.

In [540]:
recipes['reviewCount'] = recipes['reviewCount'].astype('int')

In [541]:
recipes['recipeID'] = recipes['recipeID'].astype('int')

In [542]:
recipes['recipeName'] = recipes['recipeName'].astype('str')

In [543]:
recipes['recipePhoto'] = recipes['recipePhoto'].astype('str')

In [544]:
recipes['author'] = recipes['author'].astype('str')

In [545]:
recipes['ingredients'] = recipes['ingredients'].astype('str')

In [546]:
recipes['directions'] = recipes['directions'].astype('str')

In [552]:
recipes['prepareTime_min'] = recipes['prepareTime_min'].astype('str')

In [553]:
recipes['cookTime_min'] = recipes['cookTime_min'].astype('str')

In [554]:
recipes['totalTime_min'] = recipes['totalTime_min'].astype('str')

In [ ]:
recipes.head(50)

In [556]:
recipes.dtypes

recipeName         object
recipeID            int64
reviewCount         int64
recipePhoto        object
author             object
prepareTime_min    object
cookTime_min       object
totalTime_min      object
ingredients        object
directions         object
dtype: object

## Cleaning Reviews

#### Read file
Replace missing values with 'nan'

In [434]:
missing_values = ["n/a", "na", "--", "Nan", "NAN", "NA", "na", "X", "", "nan", "N/A"]

In [435]:
reviews = pd.read_csv('reviews.csv', encoding='latin-1', sep=';', quotechar='\'', na_values=missing_values )

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
reviews.head(10)

In [437]:
reviews.dtypes

RecipeID     object
profileID    object
Rate         object
Comment      object
dtype: object

In [438]:
reviews.columns

Index(['RecipeID', 'profileID', 'Rate', 'Comment'], dtype='object')

In [439]:
reviews = reviews.rename(columns={"RecipeId": "recipeId", "profileID": "profileID", "Rate": "rate", "Comment": "comment"})

In [440]:
reviews.size

6315596

#### For profileID check which rows have non-numeric values

In [443]:
reviews['filter'] = list(map(lambda x: not x.isdigit(), reviews['profileID'].astype('str')))

In [447]:
reviews[reviews['filter']]

,RecipeID,profileID,rate,comment,filter


##### Drop the rows with non-numeric values

In [446]:
# get names of indexes for which 
# column Age has value 21 
index_names = reviews[ reviews['filter'] == 1 ].index 
  
# drop these row indexes 
# from dataFrame 
reviews.drop(index_names, inplace = True) 

#### Other columns do not have any non-numeric values so convert all of them to int. (conversion to str first is necessary)

In [458]:
reviews['RecipeID'] = reviews['RecipeID'].astype('str')

In [ ]:
reviews['RecipeID'] = reviews['RecipeID'].astype('int') 

In [456]:
reviews['rate'] = reviews['rate'].astype('str')

In [463]:
reviews['rate'] = reviews['rate'].astype('int')

In [461]:
reviews['comment'] = reviews['comment'].astype('str')

##### delete the filter column

In [466]:
del reviews['filter']

In [ ]:
reviews

#### Create cleaned csv files

In [557]:
recipes.to_csv("cleaned-recipes2.csv")

In [559]:
reviews.to_csv("cleaned_reviews2.csv")